## Strava Bike Mapping


In [1]:
import folium
import pandas as pd
import numpy as np
import plotly.graph_objects as go

strava = pd.read_csv("strava.csv")
strava['Date'] = pd.DatetimeIndex(strava['timestamp']).date
strava['month'] = pd.DatetimeIndex(strava['timestamp']).month
#Focus only on visualizing the client's biking workouts
strava_bikes=strava[strava['month'] >=9]

In [2]:
#drop NA data
strava_bikes["position_lat_degrees"] = strava_bikes["position_lat"] * ( 180 / 2**31 ) #convert radians to degrees
strava_bikes["position_long_degrees"] = strava_bikes["position_long"] * ( 180 / 2**31 )
strava_bikes = strava_bikes.dropna(subset=['position_lat_degrees', 'position_lat_degrees'], how = 'any')


In [3]:
m=folium.Map(location=[42.282713,-83.753], #set starting location of the map
             tiles = 'Stamen terrain', #change tile type from default
             zoom_start=13, #set starting zoom
             min_zoom=12) # set the zoom limit, as no one should be zooming out past this scale

In [4]:
#create a list of colors for our bike rides
colors = [
    'red',
    'blue',
    'cadetblue',
    'darkred',
    'lightred',
    'orange',
    'beige',
    'green',
    'darkgreen',
    'lightgreen',
    'darkblue',
    'lightblue',
    'purple',
]

The next cell will iterate through the unique workouts of our biking workouts, and create markers and lines for each of the rides. There is also code in each iteration that takes statistics such as average heart rate, average speed, average distance, and date. This information feeds in to the tooltip and the popups.

In [5]:
for i, workout in enumerate(strava_bikes['datafile'].unique()):
    # Create a temporary dataset of a unique biking workout
    current_workout = strava_bikes[strava_bikes['datafile'] == workout]
    
    # create a color from the colors list above
    # A modulo will prevent this code from erroring out in the occurrence of additional data.
    current_color = colors[i % 13]
    
    # current workout distance by taking the distance of the final row, as distance is cumulative per workout
    cwd = round(current_workout["distance"].iloc[-1],0)
    
    # take the current date of the workout. Can use min or max interchangably 
    # only time it will matter is if you want the first date of a midnight workout, or a second
    cwdate = current_workout["Date"].min()
    
    # drop heart_rate NA values
    heart = current_workout.dropna(subset=['heart_rate'], how = 'any')
    
    # round average heart rate
    heart = round(heart["heart_rate"].mean(),2)
    
    # round average speed (m/s)
    speed = round(current_workout["enhanced_speed"].mean(),2)

    folium.Marker([current_workout["position_lat_degrees"].iloc[0],current_workout["position_long_degrees"].iloc[0]], # mark the beginning of a workout, or the first index
                  popup=f'Total_distance: {cwd} m<br> heart_rate: {heart} bpm<br> avg_speed: {speed} m/s', # put the information calculated above into the popup
                  tooltip = f'Workout on {cwdate}', # show the date of the workout as the tooltip
                  icon = folium.Icon(icon = 'play-circle', color = current_color)).add_to(m) # choose an appropriate "go" icon with our current color
    
    #Folium has thousands of other icons at: https://fontawesome.com/v5.15/icons?d=gallery&p=2
    
    folium.Marker([current_workout["position_lat_degrees"].iloc[-1],current_workout["position_long_degrees"].iloc[-1]], # mark the end of a workout, or the last index
                  popup=f'Total_distance: {cwd} m<br> heart_rate: {heart} bpm<br> avg_speed: {speed} m/s', # put the information calculated above into the popup
                  tooltip = f'Workout on {cwdate}',  # show the date of the workout as the tooltip
                  icon=folium.Icon(icon = 'stop', color = current_color)).add_to(m)  # choose an appropriate "stop" icon with our current color
    
    # Create a line with a zip of the latitude and longitude degrees of our current workout
    route=folium.PolyLine(locations=zip(current_workout["position_lat_degrees"],current_workout["position_long_degrees"]), 
                          weight=4, # change the thickness of the line
                          smooth_factor=2, # smooth_factor simplifies the Polyline on each zoom increment
                          color=current_color).add_to(m) # use our current color 


In [6]:
display(m)

The above visualization allows us to explore the data on a map. Prior to putting it on a map, humans are almost entirely unable to picture where something may be, solely based on latitude and longitude coordinates. This map shows the start and stopping points of every bike ride that our client has done, and color codes each ride as well. The client is able to both hover over and click on the markers to see valuable information about that specific ride. Limitations of this visualization arise as repeated routes are taken, as the lines and markers can overlap very easily. 